In [1]:
import os
if not os.path.exists('dataset'):
    !unzip /gemini/data-1/Fatigue.zip

In [2]:
def rename_folder(dataset_path, old_name, new_name):
    jpeg_images_folder = os.path.join(dataset_path, old_name)
    if os.path.exists(jpeg_images_folder):
        images_folder = os.path.join(dataset_path, new_name)
        os.rename(jpeg_images_folder, images_folder)
        print(f"Folder '{old_name}' has been renamed to '{new_name}'.")
    else:
        print(f"Folder '{old_name}' does not exist.")

dataset_path = 'dataset/'
rename_folder(dataset_path, 'JPEGImages', 'images')

In [3]:
from ultralytics import YOLO
from glob import glob
from sklearn.model_selection import train_test_split
import yaml
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [4]:
# 获取所有图像文件和标注文件路径
image_files = glob(os.path.join(dataset_path, 'images', '*.jpg'))
annotation_files = glob(os.path.join(dataset_path, 'Annotations', '*.xml'))
print(len(image_files))

In [5]:
# 处理没有标签的图片
for pic in image_files:
    anno_name = (pic.split('.')[0] + '.xml').replace('images','Annotations')
    if anno_name not in annotation_files:
        os.remove(os.path.join(pic))

In [6]:
# 重置所有图像文件和标注文件路径并查看处理后的数据集的数量
image_files = glob(os.path.join(dataset_path, 'images', '*.jpg'))
annotation_files = glob(os.path.join(dataset_path, 'Annotations', '*.xml'))
print(len(image_files),len(annotation_files))

In [7]:
# 转换VOC格式数据为YOLO格式
import xml.etree.ElementTree as ET

classes = ['closed_eye', 'open_eye','closed_mouth','open_mouth']
# 初始化每个类别的计数
nums = [0] * len(classes)
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(image_id):
    in_file = open(dataset_path+'Annotations/%s.xml' % (image_id))
    out_file = open(dataset_path+'labels/%s.txt' % (image_id), 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        nums[cls_id] += 1
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
             float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

if not os.path.exists(dataset_path+'labels/'):
    os.makedirs(dataset_path+'labels/')
for filename in annotation_files:
    if filename.endswith('.xml'):
        # file_path = os.path.join(path, filename)
        image_id = os.path.splitext(os.path.basename(filename))[0]
        convert_annotation(image_id)

In [8]:
# 数据集分布
import matplotlib.pyplot as plt
print(classes,nums)
# 数据集标签和对应的数量
sizes = nums

# 生成饼状图
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=classes, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # 使饼图为圆形
plt.title('数据集分布')
plt.show()

In [9]:
# 进行8:2分割
labels_files = glob(os.path.join(dataset_path, 'labels', '*.txt'))
train_images, val_images, train_labels, val_labels = train_test_split(
    image_files, labels_files, test_size=0.2, random_state=42)

In [10]:
# 将文件路径写入文本文件
dataset_path = 'datasets/'

if not os.path.exists(dataset_path+'paths/'):
    os.makedirs(dataset_path+'paths/')
def write_file_list(file_list, filename):
    filename = os.path.join(dataset_path,'paths', filename)
    with open(filename, 'w') as f:
        for file in file_list:
            f.write(f"{file}\n")

write_file_list(train_images, 'train_images.txt')
write_file_list(val_images, 'val_images.txt')
write_file_list(train_labels, 'train_labels.txt')
write_file_list(val_labels, 'val_labels.txt')

In [11]:
# 生成my_det_data.yaml文件
data = {
    'train': 'paths/train_images.txt',
    'val': 'paths/val_images.txt',
    'nc': 4,  
    'names': ['closed_eye', 'open_eye','closed_mouth','open_mouth']
}

with open('Fatigue_det_data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)


In [12]:
# Load a model
model = YOLO("yolov8n.yaml") 

In [13]:
# 优化参数方案二：增加数据增强、轮数、batch
results = model.train(data="Fatigue_det_data.yaml", 
                      epochs=30, 
                      imgsz=640, 
                      batch=16,
                      workers=2,
                      mosaic=0.7,  # 增加mosaic数据增强
                      mixup=0.3,   # 增加mixup数据增强
                      copy_paste=0.2)  # 增加copy-paste数据增强

In [15]:
# 如果要导出结果，可压缩实际生成的trrain文件夹，导出压缩包
! zip -r /gemini/code/runs/detect/train3.zip /gemini/code/runs/detect/train3